# 2.2.4 Least Mean Squares
#### CS 6350: HW 2

---

[22 points] We will implement the LMS method for a linear regression task. The dataset
is from UCI repository (https://archive.ics.uci.edu/ml/datasets/Concrete+Slump+Test). The task is to predict the real-valued SLUMP of the concrete, with 7 features.
The features and output are listed in the file “concrete/data-desc.txt”. The training data are stored in the file “concrete/train.csv”, consisting of 53 examples. The
test data are stored in “concrete/test.csv”, and comprise of 50 examples. In both the
training and testing datasets, feature values and outputs are separated by commas.


In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline  
#plt.rcParams['figure.figsize'] = (10, 6)

In [2]:
test_data = pd.read_csv('concrete/test.csv', header=None)
train_data = pd.read_csv('concrete/train.csv', header=None)

# first 7 columns are features, last column (Slump) is output
columns = ['Cement', 'Slag', 'Fly ash', 'Water','SP', 'Coarse Aggr', 'Fine Aggr', 'Slump']
features = columns[:-1]
output = columns[-1]

test_data.columns = columns
train_data.columns = columns

In [3]:
train_data.head()

,Cement,Slag,Fly ash,Water,SP,Coarse Aggr,Fine Aggr,Slump
0,-0.890447,-1.295950,1.188030,-1.450430,1.238500,1.876070,-0.818684,-0.005574
1,0.549165,0.066917,-0.517804,0.638098,0.164716,0.227616,-0.945600,0.568593
2,0.941555,-1.292630,1.068030,1.950890,-0.085833,-1.180960,-0.385584,0.798260
3,-0.966886,1.047520,0.376289,1.831540,0.880569,-0.988829,-0.898007,0.568593
4,0.638344,0.233121,-0.364867,0.488917,0.164716,-0.011126,-0.961465,0.855677


In [4]:
train_data.shape, test_data.shape

((53, 8), (50, 8))

In [5]:
A = train_data.iloc[:,:-1].to_numpy()

In [6]:
A.shape

(53, 7)

In [2]:
import numpy as np

In [3]:
np.log(10)

2.302585092994046

In [5]:
np.log(100)

4.605170185988092

---

### A. Batch Gradient Descent

 [8 points] Implement the batch gradient descent algorithm, and tune the learning
rate r to ensure the algorithm converges. To examine convergence, you can watch
the norm of the weight vector difference, kwt−wt−1k, at each step t. if kwt−wt−1k
is less than a tolerance level, say, 1e − 6, you can conclude that it converges. You
can initialize your weight vector to be 0. Please find an appropriate r such that
the algorithm converges. To tune r, you can start with a relatively big value,
say, r = 1, and then gradually decrease r, say r = 0.5, 0.25, 0.125, . . ., until you
see the convergence. Report the learned weight vector, and the learning rate r.
Meanwhile, please record the cost function value of the training data at each step,
and then draw a figure shows how the cost function changes along with steps. Use
your final weight vector to calculate the cost function value of the test data.


In [7]:
def bdc_method(A,b, epsilon, t=0.01):
    """ Batch Gradient Descent Method
    
    Args:
        A (mxn numpy array): input array holding m samples with n features
        b (mx1 numpy array): output 
        epsilon (float): tolerance level
        t (float): learning rate

    Returns:
        tree (dict): dictionary structure represented the decision tree
  
    """
    x = np.zeros(A.shape[1])
    
    diff = 1

    iter=0
    fun_val=f(A,b,x)
    fun_history = fun_val
    
    grad=g(A,b,x)
    while (np.linalg.norm(diff)>epsilon):
        iter=iter+1
        
        # break out of while loop if diverging
        if np.linalg.norm(diff)>1e20:
            break
  
        # define new point x = x + t d, d = - grad
        x_new=x-t*grad
        diff = x_new - x
        x = x_new

        fun_val= f(A,b,x)
        fun_history = np.vstack((fun_history,fun_val))
        grad = g(A,b,x)
        #if iter % 100 == 0:
            #print('iter_number = {}, tol = {:.4e}, fun_val = {:.4e}'.format(iter, np.linalg.norm(diff), fun_val))
        
    if np.linalg.norm(diff)>1e20:
        iter = None
        print('Algorithm does not converge!')
    print('iter_number = {}, tol = {:.4e}, fun_val = {:.4e}'.format(iter, np.linalg.norm(diff), fun_val))
    return x, fun_history

In [8]:
def f(A,b, x):
    """ Calculates the loss funtion value
    """
    val = 0.0
    
    for i in range(A.shape[0]):
        val += (b[i] - x@A[i,:])**2
        
    return val / 2

In [9]:
def g(A,b, x):
    """ Calculates the gradient
    """
    grad = np.zeros(x.shape)
    
    for j in range(len(grad)):
        for i in range(A.shape[0]):
            grad[j] += (b[i] - x@A[i,:])*A[i,j]
        
    return - grad

### Learning weights

In [10]:
train_data_array = train_data.to_numpy()

A = train_data_array[:,:-1]
A = np.insert(A,0,np.ones(A.shape[0]), axis=1)

b = train_data_array[:,-1]

In [10]:
A.shape, b.shape

((53, 8), (53,))

In [11]:
epsilon = 1e-6

In [38]:
x, fh = gradient_method(A,b, epsilon)

In [40]:
fh.shape

(7095, 1)

In [54]:
plt.scatter(range(len(fh)),fh, s=3)
plt.title('Cost function vs Time')
plt.xlabel('Iteration #')
plt.ylabel('Cost Function Value')

plt.show()

### Tuning learning rate

In [106]:
rates = [1/(2**i) for i in range(8)]

rates

[1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125]

In [62]:
4%2

0

In [107]:
x1, fh1 = batch_gradient_method(A,b, epsilon,t=1)

Algorithm does not converge!
iter_number = None, tol = 2.1795e+20, fun_val = 3.1189e+42


In [108]:
x2, fh2 = batch_gradient_method(A,b, epsilon,t=0.5)

Algorithm does not converge!
iter_number = None, tol = 8.5691e+20, fun_val = 4.7487e+43


In [109]:
x3, fh3 = batch_gradient_method(A,b, epsilon,t=0.25)

Algorithm does not converge!
iter_number = None, tol = 1.8467e+20, fun_val = 2.1388e+42


In [110]:
x4, fh4 = batch_gradient_method(A,b, epsilon,t=0.125)

Algorithm does not converge!
iter_number = None, tol = 4.5099e+20, fun_val = 1.1979e+43


In [111]:
x5, fh5 = batch_gradient_method(A,b, epsilon,t=0.0625)

Algorithm does not converge!
iter_number = None, tol = 6.3839e+20, fun_val = 2.1036e+43


In [112]:
x6, fh6 = batch_gradient_method(A,b, epsilon,t=0.03125)

Algorithm does not converge!
iter_number = None, tol = 1.8178e+20, fun_val = 1.2720e+42


In [113]:
x7, fh7 = batch_gradient_method(A,b, epsilon,t=0.015625)

Algorithm does not converge!
iter_number = None, tol = 1.0038e+20, fun_val = 1.8157e+41


In [114]:
x8, fh8 = batch_gradient_method(A,b, epsilon,t=0.0078125)

iter_number = 8800, tol = 9.9976e-07, fun_val = 1.4982e+01


In [115]:
x9, fh9 = batch_gradient_method(A,b, epsilon,t=0.01)

iter_number = 7094, tol = 9.9916e-07, fun_val = 1.4982e+01


Use your final weight vector to calculate the cost function value of the test data:

In [121]:
# cost function on training data
fh9[-1]

array([14.98194366])

In [118]:
test_data_array = test_data.to_numpy()

A_test = test_data_array[:,:-1]
A_test = np.insert(A_test,0,np.ones(A_test.shape[0]), axis=1)

b_test = test_data_array[:,-1]

In [122]:
# cost funtion on test data
f(A_test,b_test, x9)

23.36175759033997

---

### B. Stochastic Gradient Descent

 [8 points] Implement the stochastic gradient descent (SGD) algorithm. You can
initialize your weight vector to be 0. Each step, you randomly sample a training
example, and then calculate the stochastic gradient to update the weight vector.
Tune the learning rate r to ensure your SGD converges. To check convergence, you
can calculate the cost function of the training data after each stochastic gradient
update, and draw a figure showing how the cost function values vary along with
the number of updates. At the beginning, your curve will oscillate a lot. However,
with an appropriate r, as more and more updates are finished, you will see the
cost function tends to converge. Please report the learned weight vector, and the
learning rate you chose, and the cost function value of the test data with your
learned weight vector.

(53, 8)

In [23]:
def sgd_method(df, attrs, target_attr, epsilon, t=0.01):
    """ Stochastic Gradient Descent Method
    
    Args:
        A (mxn numpy array): input array holding m samples with n features
        b (mx1 numpy array): output 
        epsilon (float): tolerance level
        t (float): learning rate

    Returns:
        tree (dict): dictionary structure represented the decision tree
  
    """
    
    m,n = df.shape
    
    # convert to numpy array
    data = df.to_numpy()
    
    # Separate to Ax=b where A is input matrix, x is vector of weights, and b the vector of outputs
    A = data[:,:-1]
    A = np.insert(A,0,np.ones(m), axis=1)
    b = data[:,-1] 
    x = np.zeros(n)

    iter=0
    max_iter = 20000
    cur_val=100
    prev_val = np.inf
    history = cur_val
    
    while np.linalg.norm(prev_val-cur_val) > epsilon and iter < max_iter:
        iter = iter + 1
        prev_val = cur_val
        
        # shuffle indexes for sampling
        indexes = np.random.randint(m, size=m)
        
        for i in indexes:
            # define new point x = x + t d, d = - grad
            x = x + t*(b[i] - np.dot(x,A[i]))*A[i]

        cur_val= f(A,b,x)
        history = np.vstack((history,cur_val))
            
        
        #print('i = {}, tol = {:.4e}, fun_val = {:.4e}'.format(iter, np.linalg.norm(prev_val-cur_val), cur_val))
        
    print('i = {}, tol = {:.4e}, fun_val = {:.4e}'.format(iter, np.linalg.norm(prev_val-cur_val), cur_val))
    return x, history

In [228]:
np.linalg.norm(100-0) < epsilon

False

In [223]:
epsilon

1e-06

In [13]:
data = train_data

In [56]:
x_b, h_b = sgd_method(data, features, output, epsilon,t=0.00001)

i = 3497, tol = 2.1769e-07, fun_val = 1.6266e+01


In [189]:
c = np.random.randint(len(train_data), size=len(train_data))

In [190]:
len(c)

53

In [160]:
train_data.head()

,Cement,Slag,Fly ash,Water,SP,Coarse Aggr,Fine Aggr,Slump
0,-0.890447,-1.295950,1.188030,-1.450430,1.238500,1.876070,-0.818684,-0.005574
1,0.549165,0.066917,-0.517804,0.638098,0.164716,0.227616,-0.945600,0.568593
2,0.941555,-1.292630,1.068030,1.950890,-0.085833,-1.180960,-0.385584,0.798260
3,-0.966886,1.047520,0.376289,1.831540,0.880569,-0.988829,-0.898007,0.568593
4,0.638344,0.233121,-0.364867,0.488917,0.164716,-0.011126,-0.961465,0.855677


In [147]:
d = train_data.sample(frac=1)   

In [148]:
d.head()

,Cement,Slag,Fly ash,Water,SP,Coarse Aggr,Fine Aggr,Slump
9,-0.852227,1.180480,0.493932,-0.853710,1.238500,-0.465871,0.101456,-2.072580
46,-0.903186,-1.295950,1.164500,-1.102350,1.238500,1.785120,-0.898007,0.338926
49,0.867663,0.482426,-1.753060,-0.555349,-0.909064,-0.056600,1.196110,-1.728080
7,-0.182107,0.580486,1.058620,0.076184,-0.407967,0.000243,-1.435810,1.085340
8,-1.142700,-0.551360,1.010390,-1.286330,-1.088030,1.860150,-0.564853,0.626010


In [137]:
d.shape

(53, 8)

In [ ]:
train_data.size

In [125]:
train_data.head()

,Cement,Slag,Fly ash,Water,SP,Coarse Aggr,Fine Aggr,Slump
0,-0.890447,-1.295950,1.188030,-1.450430,1.238500,1.876070,-0.818684,-0.005574
1,0.549165,0.066917,-0.517804,0.638098,0.164716,0.227616,-0.945600,0.568593
2,0.941555,-1.292630,1.068030,1.950890,-0.085833,-1.180960,-0.385584,0.798260
3,-0.966886,1.047520,0.376289,1.831540,0.880569,-0.988829,-0.898007,0.568593
4,0.638344,0.233121,-0.364867,0.488917,0.164716,-0.011126,-0.961465,0.855677


In [129]:
features

['Cement', 'Slag', 'Fly ash', 'Water', 'SP', 'Coarse Aggr', 'Fine Aggr']

In [130]:
output

'Slump'

In [ ]:
stoc_gradient_method(train_data, features, output, epsilon, t=0.01)

---

### C. Optimal weight vector with analytical form

[6 points] We have discussed how to calculate the optimal weight vector with an
analytical form. Please calculate the optimal weight vector in this way. Comparing with the weight vectors learned by batch gradient descent and stochastic
gradient descent, what can you conclude? Why?
